# Class Statistics

Importing exported, transformed dataclasses

In [2]:
import json
import matplotlib.pyplot as plt
import operator

In [3]:
with open("K:\\Pharos-all_data\\entity_dump_cleaned_and_restructured_2.json", encoding = 'cp1252') as f:
    data = json.load(f)

### Functions

In [4]:
def exchange_id_with_value(id):
    value = data[id]
    for index in range(len(data[id])):
        for index_2 in range(len(data[id][index])):
            if type(data[id][index][index_2]) == dict:
                label = data[id][index][index_2]["Label"]
                break


    return label

## Artwork / Artist Relation

collecting all works from the data

In [ ]:
work_dict = {}
for key in list(data.keys()):
    if key.__contains__("work"):
        work_dict[key] = data[key]

looking for the artist of each artwork. When there is no artist, a placeholder will be created

In [ ]:
works_and_artists = {}
for work in list(work_dict.keys()):
    for attribute in work_dict[work]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and attribute[index][0] == "PC14_carried_out_by(cidoc-crm)":
                work_as_key = work.split("(")[0]
                artist_id = attribute[index][1].split("(")[0]
            elif type(attribute[index]) == list and attribute[index][1] == "PC14_carried_out_by(cidoc-crm)":
                work_as_key = work.split("(")[0]
                artist_id = attribute[index][0].split("(")[0]

            if type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][1])
            elif type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][0])

    if work_as_key != None:
        works_and_artists[work_as_key] = (artist_id, profession_id)
    
      
    if work_as_key == None:
        work_as_key = work.split("(")[0]
        works_and_artists[work_as_key] = ["No Artist", None]

    work_as_key = None
    


In [ ]:
professions = set()
for element in works_and_artists:
    professions.add(works_and_artists[element][1])

In [ ]:
print(professions)

Iterating over each artwork and counting the occourences of artist. Counting, how many works an artist has done.

In [ ]:
artist_work_count = {}
artist_and_profession = {}
for work in list(works_and_artists.keys()):
    if works_and_artists[work][0] not in artist_work_count:
        artist_work_count[works_and_artists[work][0]] = [1, works_and_artists[work][1]]
        artist_and_profession[works_and_artists[work][0]] = works_and_artists[work][1]
    else:
        artist_work_count[works_and_artists[work][0]][0] += 1

Iterating over the artists and counting each artist with a certain amount of artworks. How many artists have made exactly one artwork, how many have made two and so on

In [ ]:
artists_with_mult_works_count = {}

for artist in list(artist_work_count.keys()):
    if artist_work_count[artist][0] not in artists_with_mult_works_count:
        artists_with_mult_works_count[artist_work_count[artist][0]] = 1
    else:
        artists_with_mult_works_count[artist_work_count[artist][0]] += 1

Transforming the dictionary data in a list of coordinates and splitting them in two eperate lists.

In [ ]:
sorted_keys = list(artists_with_mult_works_count.keys())
sorted_keys.sort()

artists_with_mult_works_count_list = []
for key in sorted_keys:
    artists_with_mult_works_count_list.append([key, artists_with_mult_works_count[key]])

works_count = [element[0] for element in artists_with_mult_works_count_list]
artist_count = [element[1] for element in artists_with_mult_works_count_list]

ploting the data

In [ ]:
plt.figure(figsize=(25,25))
plt.yscale('log')
plt.xscale('log')
plt.scatter(works_count, artist_count)
#plt.axis([0,100,0,100])
plt.title("Artwork / Artist Relation")
plt.xlabel("number of artworks")
plt.ylabel("number of artists")

In [ ]:
most_works = [element[0] for element in list(artist_work_count.values())]
most_works.sort(reverse=True)
most_works = most_works[1:11]

In [ ]:
top_artists = [element for element in artist_work_count if artist_work_count[element][0] in most_works]
top_artists = [element + "(actor)" for element in top_artists]
top_artists = [exchange_id_with_value(element) for element in top_artists]

In [ ]:
print("Artists with most artworks")
for element in top_artists:
    print("{}".format(element))

### Attempt to scatter in color according to profession

In [ ]:
artist_work_count_and_profession_list = []

for element in artist_work_count:
    artist_work_count_and_profession_list.append([element, artist_work_count[element][0], artist_work_count[element][1]])

In [ ]:
def takeSecond(elem):
    return elem[1]

artist_work_count_and_profession_list.sort(key=takeSecond)

In [ ]:
artist_list = [element[0] for element in artist_work_count_and_profession_list]
count_list = [element[1] for element in artist_work_count_and_profession_list]
profession_list = [element[2] for element in artist_work_count_and_profession_list]

In [ ]:
id = 0
profession_list_numberized = []
helping_dict = {}
for element in profession_list:
    if element not in helping_dict:
        helping_dict[element] = id
        id += 1
        profession_list_numberized.append(helping_dict[element])
    else:
        profession_list_numberized.append(helping_dict[element])

In [ ]:
print(list(helping_dict.keys()))

In [ ]:
import numpy as np

plt.figure(figsize=(25,25))
plt.scatter(artist_list, count_list, c=profession_list_numberized)
#plt.axis([5250,5650,0,200])
plt.xlabel("artists")
plt.ylabel("number of artworks")

cb = plt.colorbar()
loc = np.arange(0, max(profession_list_numberized), max(profession_list_numberized)/float(len(list(helping_dict.keys()))))
cb.set_ticks(loc)
cb.set_ticklabels(profession_list)

## Artwork / Location Relation

collecting all works from the data

In [ ]:
work_dict = {}
for key in list(data.keys()):
    if key.__contains__("work"):
        work_dict[key] = data[key]

In [ ]:
works_and_places = {}
for work in list(work_dict.keys()):
    for attribute in work_dict[work]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and attribute[index][0] == "P55_has_current_location(cidoc-crm)":
                work_as_key = work.split("(")[0]
                place_id = attribute[index][1].split("(")[0]
            elif type(attribute[index]) == list and attribute[index][1] == "P55_has_current_location(cidoc-crm)":
                work_as_key = work.split("(")[0]
                place_id = attribute[index][0].split("(")[0]
            """
            if type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][1])
            elif type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][0])
                """

    if work_as_key != None:
        works_and_places[work_as_key] = place_id
    
      
    if work_as_key == None:
        work_as_key = work.split("(")[0]
        works_and_places[work_as_key] = "No Place"

    work_as_key = None
    


In [ ]:
place_work_count = {}
for work in list(works_and_places.keys()):
    if works_and_places[work] not in place_work_count:
        place_work_count[works_and_places[work]] = 1
    else:
        place_work_count[works_and_places[work]] += 1

In [ ]:
places_with_mult_works_count = {}

for place in list(place_work_count.keys()):
    if place_work_count[place] not in places_with_mult_works_count:
        places_with_mult_works_count[place_work_count[place]] = 1
    else:
        places_with_mult_works_count[place_work_count[place]] += 1

In [ ]:
sorted_keys = list(places_with_mult_works_count.keys())
sorted_keys.sort()

places_with_mult_works_count_list = []
for key in sorted_keys:
    places_with_mult_works_count_list.append([key, places_with_mult_works_count[key]])

works_count = [element[0] for element in places_with_mult_works_count_list]
places_count = [element[1] for element in places_with_mult_works_count_list]

In [ ]:
plt.figure(figsize=(25,25))
plt.scatter(works_count, places_count)
plt.yscale('log')
plt.xscale('log')
plt.title("Artwork / Location Relation")
plt.xlabel("number of works")
plt.ylabel("number of places")

In [ ]:
most_works = [element for element in list(place_work_count.values())]
most_works.sort(reverse=True)
most_works = most_works[1:11]

In [ ]:
top_places = [element for element in place_work_count if place_work_count[element] in most_works]
top_places = [element + "(place)" for element in top_places]
top_places = [exchange_id_with_value(element) for element in top_places]

In [ ]:
print("Places with most artworks")
for element in top_places:
    print("{}".format(element))

## Person / Location Relation

In [ ]:
person_dict = {}
for key in list(data.keys()):
    if key.__contains__("actor"):
        person_dict[key] = data[key]

In [ ]:
persons_and_places = {}
person_as_key = None
for person in list(person_dict.keys()):
    for attribute in person_dict[person]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and attribute[index][0] == "P74_has_current_or_former_residence(cidoc-crm)":
                person_as_key = person.split("(")[0]
                place_id = attribute[index][1].split("(")[0]
            elif type(attribute[index]) == list and attribute[index][1] == "P74_has_current_or_former_residence(cidoc-crm)":
                person_as_key = person.split("(")[0]
                place_id = attribute[index][0].split("(")[0]
            """
            if type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][1])
            elif type(attribute[index]) == list and attribute[index][0] == "P14.1_in_the_role_of(cidoc-crm)":
                profession_id = exchange_id_with_value(attribute[index][0])
                """

    if person_as_key != None:
        persons_and_places[person_as_key] = place_id
    
      
    if person_as_key == None:
        person_as_key = person.split("(")[0]
        persons_and_places[person_as_key] = "No Place"

    person_as_key = None
    


In [ ]:
place_person_count = {}
for person in list(persons_and_places.keys()):
    if persons_and_places[person] not in place_person_count:
        place_person_count[persons_and_places[person]] = 1
    else:
        place_person_count[persons_and_places[person]] += 1

In [ ]:
places_with_mult_persons_count = {}

for place in list(place_person_count.keys()):
    if place_person_count[place] not in places_with_mult_persons_count:
        places_with_mult_persons_count[place_person_count[place]] = 1
    else:
        places_with_mult_persons_count[place_person_count[place]] += 1

In [ ]:
sorted_keys = list(places_with_mult_persons_count.keys())
sorted_keys.sort()

places_with_mult_persons_count_list = []
for key in sorted_keys:
    places_with_mult_persons_count_list.append([key, places_with_mult_persons_count[key]])

persons_count = [element[0] for element in places_with_mult_persons_count_list]
places_count = [element[1] for element in places_with_mult_persons_count_list]

In [ ]:
plt.figure(figsize=(25,25))
plt.scatter(persons_count, places_count)
plt.yscale('log')
plt.xscale('log')
#plt.axis([0,250,0,250])
plt.title("Person / Location Relation")
plt.xlabel("number of persons")
plt.ylabel("number of places")

In [ ]:
most_persons = [element for element in list(place_person_count.values())]
most_persons.sort(reverse=True)
most_persons = most_persons[1:11]

In [ ]:
top_places = [element for element in place_person_count if place_person_count[element] in most_persons]
top_places = [element + "(place)" for element in top_places]
top_places = [exchange_id_with_value(element) for element in top_places]

In [ ]:
print("Places with most Artists")
for element in top_places:
    print("{}".format(element))

## Photographer / Artwork Relation

### Preprocessing

In [5]:
work_dict = {}
for key in list(data.keys()):
    if key.__contains__("work"):
        work_dict[key] = data[key]

In [6]:
actor_dict = {}
for key in list(data.keys()):
    if key.__contains__("actor"):
        actor_dict[key] = data[key]

In [7]:
type_dict = {}
for key in list(data.keys()):
    if key.__contains__("type"):
        type_dict[key] = data[key]

In [8]:
photo_dict = {}
for key in list(data.keys()):
    if key.__contains__("photo"):
        photo_dict[key] = data[key]

In [9]:
photos_with_works = {}
for counter, photo in enumerate(photo_dict):
    for attribute in photo_dict[photo]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and "P128_carries(cidoc-crm)" in attribute[index]:
                photos_with_works[photo] = photo_dict[photo]
            """
            if type(attribute[index]) == dict:
                print(attribute[index])
            """

In [11]:
print(len(photo_dict))

print(len(photos_with_works))

45192
33029


In [12]:
print(len(photo_dict))

print(len(photos_with_works))

45192
33029


In [19]:
photographer_dict = {}

for counter, photo in enumerate(photo_dict):
    for attribute in photo_dict[photo]:
        for index in range(len(attribute)):
            """
            if type(attribute[index]) == list and "P128_carries(cidoc-crm)" in attribute[index]:
                work_on_photo = [ element.split("(")[1].replace(")", "") for element in attribute[index] if element.__contains__("visual_item")][0]
            """
            if type(attribute[index]) == list and "P14_carried_out_by(cidoc-crm)" in attribute[index]:
                photographer = exchange_id_with_value([element for element in attribute[index] if element.__contains__("actor")][0])

    if photographer not in photographer_dict:
        photographer_dict[photographer] = 1
    else:
        photographer_dict[photographer] += 1

In [24]:
print(photographer_dict["Bazzechi,Ivo"])

3196


In [10]:
photographer_dict = {}

for counter, photo in enumerate(photos_with_works):
    for attribute in photo_dict[photo]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and "P128_carries(cidoc-crm)" in attribute[index]:
                work_on_photo = [ element.split("(")[1].replace(")", "") for element in attribute[index] if element.__contains__("visual_item")][0]
            if type(attribute[index]) == list and "P14_carried_out_by(cidoc-crm)" in attribute[index]:
                photographer = exchange_id_with_value([element for element in attribute[index] if element.__contains__("actor")][0])

    if photographer not in photographer_dict:
        photographer_dict[photographer] = {work_on_photo: 1}
    elif work_on_photo not in photographer_dict[photographer]:
        photographer_dict[photographer][work_on_photo] = 1
    else:
        photographer_dict[photographer][work_on_photo] += 1

In [30]:
print(len(photographer_dict))
print(list(photographer_dict.keys())[0:5])
print(len(photographer_dict[list(photographer_dict.keys())[0]]))
print(photographer_dict[list(photographer_dict.keys())[0]])

68
['Sigismondi,Roberto', 'Artini,Luigi', 'Brogi', 'Fiorucci,Carlo', 'Visconti,FrancescoArese']
4049
{'07874073': 17, '07963272': 14, '07960775': 1, '07961712': 1, '07960772': 2, '07960779': 1, '07600323': 1, '07961711': 3, '07600328': 2, '07963273': 2, '07874071': 1, '07963274': 3, '07960771': 1, '07963275': 4, '07705202': 2, '07855840': 1, '07960776': 1, '07702708': 1, '07960770': 1, '07600326': 1, '07961719': 1, '07960778': 2, '07961713': 3, '07855844': 1, '07963280': 15, '07961715': 1, '07963277': 1, '07605216': 1, '07855834': 1, '70007796': 1, '07855833': 1, '07960777': 1, '07961716%2CT%2C001': 1, '70000499': 1, '70007799': 1, '07855843': 1, '70007795': 1, '07963270': 1, '70007794': 1, '70007793': 1, '07961714': 1, '07963279': 1, '70007798': 1, '70007797': 1, '07961716%2CT%2C002': 1, '07961718': 1, '07874075': 1, '70007790': 1, '07874077': 2, '07652282': 1, '70007791': 1, '70007792': 1, '07873982': 1, '07874081': 1, '07873986': 1, '07874079': 1, '07811326': 5, '07963260': 1, '0796

In [12]:
for element in list(photographer_dict.keys())[:5]:
    photographer_artwork_values = sorted(photographer_dict[element].items(), key=operator.itemgetter(1), reverse=True)
    photographer_artwork_values = dict(photographer_artwork_values)

    print("TOP 10 artworks photographed by {}\n".format(element))
    for top_position in list(photographer_artwork_values.keys())[:10]:
        print("{} times:\t{}".format(photographer_artwork_values[top_position], top_position))
    
    print("\nTotal number of artworks photographed: {}\n\n".format(len(photographer_dict[element])))

TOP 10 artworks photographed by Sigismondi,Roberto

45 times:	07705627
39 times:	07705270
38 times:	07705617
36 times:	07874132
32 times:	07963237
29 times:	70004783
27 times:	07962386
26 times:	07857019
25 times:	70004833
24 times:	07874062

Total number of artworks photographed: 4049


TOP 10 artworks photographed by Artini,Luigi

60 times:	07930037%2CT%2C001%2CT
57 times:	07580149
56 times:	07501836%2CT%2C001%2CT
55 times:	07873109
50 times:	07501836%2CT%2C001%2CT%2C002%2CT%2C001
49 times:	07873106
48 times:	70001475
44 times:	07930039%2CT%2C001
44 times:	07502889
43 times:	07930039%2CT%2C002

Total number of artworks photographed: 2834


TOP 10 artworks photographed by Brogi

10 times:	07580181
8 times:	07599766
8 times:	07605036
8 times:	07580319
6 times:	07600337
6 times:	07580166
6 times:	07580216
5 times:	07600318
5 times:	07858369
5 times:	07580277

Total number of artworks photographed: 146


TOP 10 artworks photographed by Fiorucci,Carlo

16 times:	07651637
15 times:	0750222

In [34]:
print(len(photographer_dict))

68


In [13]:
work_photographer_relation_dict = {}

for photographer in photographer_dict:
    for work in photographer_dict[photographer]:
        if work not in work_photographer_relation_dict:
            work_photographer_relation_dict[work] = {photographer: photographer_dict[photographer][work]}
        else:
            work_photographer_relation_dict[work][photographer] = photographer_dict[photographer][work]

Question: How many artworks are photographed by multiple photographers?

In [14]:
counter = 0
for work in work_photographer_relation_dict:
    if len(work_photographer_relation_dict[work]) > 1:
        counter += 1

print("Answer: {} artworks were photographed by multiple photographers".format(counter))

Answer: 688 artworks were photographed by multiple photographers


Question: Which artwork was photographed by the most photographers?

In [15]:
#Answer Part 1

counter = 0
for work in work_photographer_relation_dict:
    if ((len(work_photographer_relation_dict[work]) > 1) and (len(work_photographer_relation_dict[work]) > counter)):
        counter = len(work_photographer_relation_dict[work])

print("The artworks with the most photographers were photographed by {} diffrent persons".format(counter))


The artworks with the most photographers were photographed by 8 diffrent persons


In [16]:
#Answer Part 2

works = []
for work in work_photographer_relation_dict:
    if len(work_photographer_relation_dict[work]) == 8:
        works.append(work)

print("Answer: {} artworks were photographed by {} photographers. These are the IDs: {}".format(len(works), counter, works))

Answer: 2 artworks were photographed by 8 photographers. These are the IDs: ['07930087', '07501527']


In [17]:
print(work_photographer_relation_dict[works[0]])

{'Sigismondi,Roberto': 6, 'Artini,Luigi': 2, 'Brogi': 1, 'Lotz-Bauer,Hilde': 1, 'Polesel,Mario': 2, 'Schmidt,HansWerner': 1, 'Bazzechi,Ivo': 2, 'Trachtenberg,Marvin': 1}


In [18]:
quantity_work_photographer_dict = {}

for work in work_photographer_relation_dict:
    if len(work_photographer_relation_dict[work]) not in quantity_work_photographer_dict:
        quantity_work_photographer_dict[len(work_photographer_relation_dict[work])] = 1
    else:
        quantity_work_photographer_dict[len(work_photographer_relation_dict[work])] += 1

In [19]:
sorted_key_list = list(quantity_work_photographer_dict.keys())
sorted_key_list.sort(reverse=True)
# print(sorted_key_list)

for key in sorted_key_list:
    print("Number of Artworks which are photographed by {} photographers: {}".format(key, quantity_work_photographer_dict[key]))

Number of Artworks which are photographed by 8 photographers: 2
Number of Artworks which are photographed by 7 photographers: 1
Number of Artworks which are photographed by 6 photographers: 2
Number of Artworks which are photographed by 5 photographers: 15
Number of Artworks which are photographed by 4 photographers: 37
Number of Artworks which are photographed by 3 photographers: 95
Number of Artworks which are photographed by 2 photographers: 536
Number of Artworks which are photographed by 1 photographers: 13361


## Photographer / Artist Relation

In [20]:
work_dict = {}
for key in list(data.keys()):
    if key.__contains__("work"):
        work_dict[key] = data[key]

In [21]:
actor_dict = {}
for key in list(data.keys()):
    if key.__contains__("actor"):
        actor_dict[key] = data[key]

In [22]:
type_dict = {}
for key in list(data.keys()):
    if key.__contains__("type"):
        type_dict[key] = data[key]

In [23]:
photo_dict = {}
for key in list(data.keys()):
    if key.__contains__("photo"):
        photo_dict[key] = data[key]

In [24]:
photos_with_works = {}
for counter, photo in enumerate(photo_dict):
    for attribute in photo_dict[photo]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and "P128_carries(cidoc-crm)" in attribute[index]:
                photos_with_works[photo] = photo_dict[photo]
            """
            if type(attribute[index]) == dict:
                print(attribute[index])
            """

In [25]:
photographer_dict = {}

for counter, photo in enumerate(photos_with_works):
    for attribute in photo_dict[photo]:
        for index in range(len(attribute)):
            if type(attribute[index]) == list and "P128_carries(cidoc-crm)" in attribute[index]:
                work_on_photo = [ element.split("(")[1].replace(")", "") for element in attribute[index] if element.__contains__("visual_item")][0]
            if type(attribute[index]) == list and "P14_carried_out_by(cidoc-crm)" in attribute[index]:
                photographer = exchange_id_with_value([element for element in attribute[index] if element.__contains__("actor")][0])

    if photographer not in photographer_dict:
        photographer_dict[photographer] = {work_on_photo: 1}
    elif work_on_photo not in photographer_dict[photographer]:
        photographer_dict[photographer][work_on_photo] = 1
    else:
        photographer_dict[photographer][work_on_photo] += 1

In [26]:
print(len(photographer_dict))
print(list(photographer_dict.keys())[0:5])
print(len(photographer_dict[list(photographer_dict.keys())[0]]))

68
['Sigismondi,Roberto', 'Artini,Luigi', 'Brogi', 'Fiorucci,Carlo', 'Visconti,FrancescoArese']
4049


In [29]:
artworks_with_artists = set()
work_not_in_data = []
photographer_artist_dict = {}
artist = None
for photographer in photographer_dict:
    photographer_artist_dict[photographer] = {}
    for photo in photographer_dict[photographer]:
        try:
            for attribute in work_dict[photo+"(work)"]:
                for index in range(len(attribute)):
                    if type(attribute[index]) == list and "P14_carried_out_by(cidoc-crm)" in attribute[index]:
                        artworks_with_artists.add(photo)
                        artist = exchange_id_with_value([element for element in attribute[index] if element.__contains__("actor")][0])

            if (artist not in photographer_artist_dict[photographer]) and artist != None:
                photographer_artist_dict[photographer][artist] = photographer_dict[photographer][photo]
            elif artist in photographer_artist_dict[photographer]:
                photographer_artist_dict[photographer][artist] += photographer_dict[photographer][photo]

        except KeyError as e:
            work_not_in_data.append(photo)
            continue
        

    break

print(len(artworks_with_artists))
print(len(work_not_in_data))

print(len(photographer_artist_dict['Sigismondi,Roberto']))
print(photographer_artist_dict)

print(photographer_dict['Sigismondi,Roberto']["07705617"])

77
273
59
{'Sigismondi,Roberto': {'Campi,Bernardino(1522)': 44, 'Crespi,Daniele': 142, 'Garofalo': 137, 'Mazzucco,Giovanni': 82, 'Papa,Carlo': 43, 'Bianchi,Isidoro': 35, 'Guido(1188)': 137, 'Loo,Jean-Baptistevan': 21, 'Bibiena,Ferdinando': 6, 'Torricelli,AntonioMaria': 94, 'Gaggini,Domenico': 768, 'Fontana,Carlo(1638)': 37, 'Borlone,Giacomo': 434, 'Volpato,GiovanniBattista': 11, 'Bassano,Leandro': 143, 'Damini,Pietro': 31, 'Fogolino,Marcello': 1, 'Memling,Hans': 81, 'Valdambrino,Francescodi': 165, 'Sensi,BattistadiCristofanello': 84, 'Peruzzi,Baldassare': 54, 'Vasari,Giorgio': 378, 'Vaccaro,DomenicoAntonio': 644, 'Salimbeni,Ventura': 148, 'Santagostino,Agostino': 37, 'ArcangelodiColadaCamerino': 124, 'CimadaConegliano,GiovanniBattista': 162, 'Sangallo,Antonioda(derÄltere)': 231, "Robertod'Oderisio": 5, 'Gamberucci,Cosimo': 427, 'CristoforodiBindoccio': 20, 'AntoniodaFabriano(1451)': 21, 'SanodiPietro': 47, 'MatteodaGualdo': 458, 'GirolamodiMatteodaGualdo': 35, 'Bordone,Paris': 52, 'Ama